<a href="https://colab.research.google.com/github/mvdheram/AHCVS_Notes/blob/main/06_Hierarchical_z_buffer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hierarchical z-buffer 

Paper : https://www.cs.cmu.edu/afs/cs/academic/class/15869-f11/www/readings/greene93_hierarchicalz.pdf

**What ?**
  * **Occlusion culling algorithm (visibility algorithm)** to render visible polygons 

**Why ?**
  * **To deal with aggregated occulusion**
    * **Aggregated occlusion : small little group of objects occlude**
  * **Dealing with dynamic scenes**
    * Objects constantly moving     
  * Advances over traditional z-buffer algorithm for visibility computation.

**How ?**
  * **Goal** 
    * Render only visible polygons 
  * **Idea** 
    * Front-to-back rendering of objects 
    * Conservative methods, all visible triangles are displayed
  * **Uses**  
    * Object space coherence
      * Single computation used to resolve visibility of group of objects near in space  
      * **Data structure**  
        * Octree spatial subdivision
    * Temporal coherence
      * Visibility information from one frame
can often be used to accelerate visibility computation for the next
frame.
      * **Uses** 
        * Use geometry visible in the previous frame to construct a
starting point for algorithm 
    * Image space coherence
      * Single computation used to resolve visibility of object covering group of pixel.
      * **Data structure** 
        * Z-Pyramid (augment traditional Z-buffer scan conversion with
an image-space Z pyramid)


## Ideas of HZB

* Reduction of redundencies in 
  * **Visibility query (z-values)** 
    * **Use a single query** instead of querying each and every occluded objects.
      * How ?
        * **Using spatial coherence** (if an object is hidden, neighboring objects are also hidden)
        * Cluster the objects and consider the minimum z-value of group and compute single visibility query for the group.  
    ---
    * Visibility of triangles are resolved in image space i.e. after rasterization and performing z-query 
      * Z -query : Compare depth/z-value of already rendered object to the new object to be rendered and render the new object if its visible
    ---

    * **Data structure** 
      * 3D Octree of scene with triangles stored in deepest octree cell 
        * Triangles that cannot be stored in 8 octants due to intersecting plane, are stored in parent node.
    * **Visibility query/test using single query** 
        * **How ?**
          * **Using Bounding Box (octree cell)**  
            * Test octree cell instead of going to subtree and test each triangle in leaves. Hence, instead of testing everything in octree cell, we can discard the subtree of octree cell. 
            * **BoundingBox test** 
              * If bounding box (octree cell) is visible in image space (**after rasterization of bounding box - 2D bounding box**) by comparing pixel wise z-values of bounding box with the z-values of z-buffer(z-value of rendered object). - **Costly** 
          * **Using z-pyramid**
            * **Why ?**
              * **Reduce the cost of determining the visibility** of Bounding box(octree cube), make use of **z-pyramid of image space**.
                * Possible to conclude
very quickly a large polygon is hidden, making it unnecessary to
examine the polygon pixel by pixel.
              * As bounding boxes are projected rather than triangles, its better to build hierarchy of image space z-buffers as done for octree.
            * **What ?**
              * Hierarchy of image space z-values used to fasten the visibility test of bounding box of octree cell/cube.
              * **Computation** 
                * **Recursively four adjecent z -values (square) of z-buffer in image space are combined into coarser quadrant by choosing the maximum of 4 z-values**.
                  * Start from upper left corner and choose the maximum z-value of the 4 in the quadrant
                  * Leads to 1/4 th resolution after first iteration.
                  * Recursively perform the same operation untill a we have a single z-value. 
                    * **Top (higher level) of hierarchy(pyramid) is *single z-buffer* values (*farthest/maximum distance from frustum in image space*)**
                    *  **Going down the hierarchy, finer values of z-buffer values are stored.**
                    * **Leves (lower levels) of pyramid contain the original z-buffer values** 
                  * **How to Upadte z-values in pyramid**
                    * After update of z-buffer in the lower levels (done by GPU), perform the same operation with new value and propogate upwards
            * **How visibility test?**
              1. **Calculate the minumum value per side of Boundingbox** 
                * Model the 6 sides of bounding box (octree cell) with 6x2 triangles (6 sides - cube X 2 triangles per side) => 2D BoundingBox
                * Min(BoundingBox) = smallest value (per side?) of bounding box
              2. **Project in Z-pyramid** 
                * **Project the minimum value** (per side?) of **bounding box onto finest level of z-pyramid**.
                  * **Search for finest level from lower/finer to higher/coarser of z-pyramid where the projected side fits.**
                    * Case (polygon located at center of image/ overlapping all quadrant of z-pyramid grid)
                      * Go down to next finer level recursively in the z-pyramid 
                      * Check if polygon is hidden in all overlapping quadrants
              3. **Test visibility** 
                * **Compare the z-values of projected bounding box and corresponding z-pyramid value**
                  * for each side 
                    * **if min(z-value BoundingBox) > corresponding z value of ZPyramid**
                      * *Polygon is hidden*  
               
 


**z-pyramid algorithm**

```
Isoccluded (polygon p, zpyramid zp)

if (p does not overlap zp) (base case -  bounding box not in quadrant)
  return 

Min(boundingBox) (Minimum value of bounding box polygon p to be tested)

if (min(boundingBox) >= zp.z) (visibility test)
  return true

else
  if(Zp is leaf) 
    return false (Bounding box not hidden)
  
  else (recursive case - recurse from higher level to lower level of zp) 
    return (Isoccluded (polygon p, zp.child[0]) &&
            Isoccluded (polygon p, zp.child[1]) &&
            Isoccluded (polygon p, zp.child[2]) &&
            Isoccluded (polygon p, zp.child[3]))


```

**Temporal coherence**

* Polygons visited in the last frame in z-buffer will probably be in the next frame of z-buffer
  * Fasten the rendering 
* Use a list and add all the octree cells which were visible in the last frame.
  * First render the cells in the list (visible in the last frame)
  * Add/remove the  new visible cells from list after removal  

## Rendering algorithm 

1. Traverse octree front-to-back from viewer position
  * Why front to back ?
    * To handle occlusion 
2. Check visibility 
  * For each visited octree cell with z-pyramid 
3. If visible, render polygons of the octree cell node and traverse the children 